In [1]:
import asyncio
import aiomysql
import cv2
import numpy as np
import face_recognition
from datetime import datetime, timedelta, time
from functools import lru_cache

# Dictionary to keep track of the last attendance time for each student
last_attendance_time = {}

# Create a global database connection pool
async def create_pool():
    return await aiomysql.create_pool(
        host="localhost",
        port=3306,
        user="root",
        password="",
        db="attendancedb",
        loop=asyncio.get_event_loop()
    )

@lru_cache(maxsize=None)
async def get_current_course():
    pool = await create_pool()
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            now = datetime.now()
            current_time = now.time()
            day = now.weekday()  # Monday is 0 and Sunday is 6

            sql = """
            SELECT course_code, start_time, end_time, venue FROM timetable 
            WHERE day = %s AND start_time <= %s AND end_time >= %s
            """
            await cursor.execute(sql, (day, current_time, current_time))
            result = await cursor.fetchone()

            if result:
                print(f"Current course: {result[0]}, start time: {result[1]}, end time: {result[2]}, venue: {result[3]}")
                start_time = result[1] if isinstance(result[1], time) else (datetime.min + result[1]).time()
                end_time = result[2] if isinstance(result[2], time) else (datetime.min + result[2]).time()
                return result[0], start_time, end_time, result[3]
            else:
                print("No current course found")
                return None, None, None, None

@lru_cache(maxsize=None)
async def get_expected_venue(course_code):
    pool = await create_pool()
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            sql = "SELECT venue FROM timetable WHERE course_code = %s"
            await cursor.execute(sql, (course_code,))
            result = await cursor.fetchone()
            if result:
                return result[0]

@lru_cache(maxsize=None)
async def is_student_enrolled(student_no, course_code):
    pool = await create_pool()
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            sql = "SELECT 1 FROM course_reg WHERE student_no = %s AND course_code = %s"
            await cursor.execute(sql, (student_no, course_code))
            result = await cursor.fetchone()
            return result is not None

async def mark_attendance_async(student_no, record_type, course_code, status):
    pool = await create_pool()
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            now = datetime.now()
            dtString = now.strftime('%Y-%m-%d %H:%M:%S')
            sql = "INSERT INTO attendance (date, time, type, student_no, course_code, status) VALUES (%s, %s, %s, %s, %s, %s)"
            val = (now.date(), dtString, record_type, student_no, course_code, status)
            await cursor.execute(sql, val)
            await conn.commit()
            print(f"Attendance {record_type} marked for {student_no} at {dtString} for course_code {course_code} with status {status}")
            last_attendance_time[student_no] = (now, record_type)

async def mark_absent_async(student_no, course_code):
    pool = await create_pool()
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            now = datetime.now()
            dtString = now.strftime('%Y-%m-%d %H:%M:%S')
            sql = "INSERT INTO attendance (date, student_no, course_code) VALUES (%s, %s, %s)"
            val = (now.date(), student_no, course_code)
            await cursor.execute(sql, val)
            await conn.commit()
            print(f"Absence recorded for {student_no} at {now.date()} for course_code {course_code}")

async def load_images_from_mysql():
    images = []
    student_numbers = []
    pool = await create_pool()
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            sql = "SELECT student_no, image FROM images"
            await cursor.execute(sql)

            async for student_no, image_blob in cursor:
                image_array = np.frombuffer(image_blob, dtype=np.uint8)
                img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
                if img is not None:
                    images.append(img)
                    student_numbers.append(student_no)

    return images, student_numbers

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

async def main():
    images, student_numbers = await load_images_from_mysql()
    
    if not images:
        print("No images found in the database.")
    else:
        encodeListKnown = findEncodings(images)
        print('Encoding Complete')

        course_code, start_time, end_time, venue = await get_current_course()
        expected_venue = await get_expected_venue(course_code) if course_code else None
        message_shown = False

        cap = cv2.VideoCapture(0)

        while True:
            success, img = cap.read()
            if not success:
                print("Failed to capture image")
                break

            imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

            facesCurFrame = face_recognition.face_locations(imgS)
            encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

            if not course_code or not expected_venue:
                if not message_shown:
                    print("No current course or expected venue found, skipping attendance marking.")
                    message_shown = True

            for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
                matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
                faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
                matchIndex = np.argmin(faceDis)

                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

                if matches[matchIndex]:
                    student_no = student_numbers[matchIndex]
                    now = datetime.now()

                    if student_no in last_attendance_time:
                        last_time, last_type = last_attendance_time[student_no]
                        if last_type == "ClockIn":
                            record_type = "ClockOut"
                        else:
                            record_type = "ClockIn"
                    else:
                        record_type = "ClockIn"

                    if student_no not in last_attendance_time or now - last_attendance_time[student_no][0] > timedelta(minutes=5):
                        if course_code and expected_venue:
                            if venue == expected_venue:
                                class_start_time = datetime.combine(datetime.today(), start_time)

                                if (now - class_start_time).total_seconds() >= -600:
                                    if now.time() <= class_start_time.time():
                                        status = "On time"
                                    elif now.time() <= (class_start_time + timedelta(minutes=10)).time():
                                        status = "Late"
                                    else:
                                        status = "Too Late"

                                    if await is_student_enrolled(student_no, course_code):
                                        await mark_attendance_async(student_no, record_type, course_code, status)
                                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                                        cv2.putText(img, f"{student_no} ({status})", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                                    else:
                                        print(f"Student {student_no} is not enrolled in course {course_code}")
                            else:
                                print(f"Incorrect venue. Expected: {expected_venue}, Scanned: {venue}")
                        else:
                            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
                            cv2.putText(img, student_no, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                else:
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
                    cv2.putText(img, "Unknown", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            cv2.imshow('Webcam', img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

        if course_code:
            # Mark absent students after the class ends
            for student_no in student_numbers:
                if student_no not in last_attendance_time and await is_student_enrolled(student_no, course_code):
                    await mark_absent_async(student_no, course_code)
        else:
            print("No current course found, skipped absence marking.")

# Run the main function
import nest_asyncio
nest_asyncio.apply()
await main()


Encoding Complete
Current course: mcs308, start time: 0:00:00, end time: 15:30:00, venue: Lab2
Student 202080163 is not enrolled in course mcs308


RuntimeError: cannot reuse already awaited coroutine